In [1]:
import pandas as pd
import numpy as np
import duckdb

## Loading GL AND PR data 

In [2]:
gl_file_excel = pd.read_csv('gl.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_18188\1320238739.py:1: DtypeWarning: Columns (6,7,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  gl_file_excel = pd.read_csv('gl.csv')


In [3]:
gl_file_excel.shape

(92046, 19)

In [4]:
pr_file_excel = pd.read_csv("pr.csv")

In [5]:
pr_file_excel.shape

(89605, 63)

In [6]:
gl_file_excel.head(1)

,MY GSTIN,Type,Supplier Name,Supplier GSTIN/UIN,Invoice No,Invoice Date,Original Invoice No,Original Invoice date,Total Invoice value,Place of Supply,Total Taxable Amount,Integrated Tax Amount,Central Tax Amount,State Tax Amount,Cess Amount,ITC Available - Cess,Remarks,HL Remarks,Final Remarks
0,29AADCH3844D1ZO,B2B,SOLUTIONS INFINI TECHNOLOGIES (I) Pvt Ltd,29AANCS0353C1ZF,NaN,NaN,NaN,NaN,2001.55,Karnataka,2001.55,13.98,173.15,173.15,NaN,NaN,Supplier GSTIN missing+Supplier Tax Invoice No...,Ignore No GST ( Journal Entry),NaN


In [7]:
# Convert to datetime
gl_file_excel['Invoice Date'] = pd.to_datetime(gl_file_excel['Invoice Date'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
gl_file_excel['Invoice Date'] = gl_file_excel['Invoice Date'].dt.strftime('%d-%m-%Y')

In [8]:
# Convert to datetime
pr_file_excel['Document Date (PR)'] = pd.to_datetime(pr_file_excel['Document Date (PR)'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
pr_file_excel['Document Date (PR)'] = pr_file_excel['Document Date (PR)'].dt.strftime('%d-%m-%Y')

In [9]:
pr_file_excel.head(1)

,ITC Claim month,Matching Key,Match Status,Match Status Description,Matching at Pan/ GSTIN,Sub-Buckets,Mismatched Fields,Mismatched Fields Count,Supplier Name,Supplier GSTIN (PR),...,Financial Year,Voucher Number,Out of Range (2B),Out of Range (PR),Claimable ITC - CGST,Claimable ITC - SGST,Claimable ITC - IGST,Claimable ITC - Cess,CT Remarks,Deferred ITC Month
0,2024 01,f5552d7f441c141c,Exact,NaN,My Gstin and Supplier Gstin Level,Actioned with No Changes,Taxable Value,1,TRINITY PANEL,29AAMFT6309F1ZV,...,2023-24,Jan'24,False,False,1737.9,1737.9,0.0,0,NaN,NaN


In [10]:
gl_file = gl_file_excel.copy()

In [11]:
pr_file = pr_file_excel.copy()

In [12]:
gl_file.shape

(92046, 19)

In [13]:
pr_file.shape

(89605, 63)

In [14]:
gl = duckdb.sql("""
              select 
                "Supplier GSTIN/UIN",
                "MY GSTIN",
                regexp_replace("Invoice No", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '')   as "Invoice No",
                "Invoice Date" as "Invoice Date",
                 string_agg(case when "Remarks" is null then '-' else "Remarks" end , '-')as "Remark",
                 string_agg(case when "HL Remarks" is null then '-' else "HL Remarks" end,'-') as "HL Remark",
                    sum(coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0)) as "IGST (GL)",
                    sum(coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0)) as "CGST (GL)",
                    sum(coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0)) as "SGST (GL)",
                    sum(
                  coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0) + 
                  coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0) +
                  coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0) 
              
              ) as "Total Tax(GL)",
             
              from gl_file
              group by 1,2,3,4
              """)

In [15]:
gl.shape

(88093, 10)

In [16]:
# Rank the rows in pr_file
pr_ranked = duckdb.sql("""
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY "Supplier GSTIN (PR)", "My GSTIN (PR)", "Document Number (PR)", "Document Date (PR)"
                        ORDER BY "Document Date (PR)") AS rn
    FROM pr_file
""")

# Separate the ranked data into rank 1 and duplicates
pr_filtered =duckdb.sql("""
  
    SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
         regexp_replace("Document Number (PR)", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '') as "Document Number (PR)",
         "Document Date (PR)" as "Document Date (PR)",
         coalesce(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "IGST (PR)",
         coalesce(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)),0) as  "CGST (PR)",
         coalesce(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "SGST (PR)",
         
         CAST(REPLACE("Total Tax (PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM pr_ranked
    WHERE rn = 1
""")

In [17]:
pr_filtered.shape

(88226, 8)

In [18]:

#Rows with common invoice numbers in both GL and PR
common_invoices = duckdb.sql("""
SELECT  DISTINCT
    gl."Invoice No" AS invoice_number
FROM  gl
inner JOIN pr_ranked pr
ON REGEXP_REPLACE(gl."Invoice No", '[^a-zA-Z0-9]', '') = REGEXP_REPLACE(pr."Document Number (PR)", '[^a-zA-Z0-9]', '')""")

In [19]:
common_invoices.shape

(71652, 1)

In [20]:

only_in_gl = duckdb.sql("""
SELECT 
    gl."Supplier GSTIN/UIN",
    gl."MY GSTIN",
    gl."Invoice No",
    gl."Invoice Date",
    gl."IGST (GL)",
    gl."CGST (GL)",
    gl."SGST (GL)",
    gl."Total Tax(GL)",
    NULL AS "Supplier GSTIN (PR)",
    NULL AS "My GSTIN (PR)",
    NULL AS "Document Number (PR)", 
    NULL AS "Document Date (PR)",
    NUll as "IGST (PR)",
    Null as "CGST (PR)",
    Null as "SGST (PR)",
    NULL AS "Total Tax(PR)", 
    "Remark" ,
    "HL Remark" ,
    NULL AS "Matching Keys",
    'Invoice present only in GL' AS Remarks,
    Null as "Remarks_1"
FROM gl AS gl
LEFT JOIN common_invoices ci
ON gl."Invoice No" = ci.invoice_number
WHERE ci.invoice_number IS NULL
""")

only_in_pr = duckdb.sql("""
SELECT 
    NULL AS "Supplier GSTIN/UIN",
    NULL AS "MY GSTIN",
    NULL AS "Invoice No", 
    NULL AS "Invoice Date", 
    NULL as "IGST (GL)",
    Null as "CGST (GL)",
    Null as "SGST (GL)",
    NULL AS "total Tax(GL)",
    pr.*,
    NULL AS "Remark",
    NULL as "HL Remark",
    NULL AS "Matching Keys",
    'Invoice present only in PR' AS Remarks,
    null as "Remarks_1"
FROM pr_filtered pr
LEFT JOIN common_invoices ci
ON pr."Document Number (PR)" = ci.invoice_number
WHERE ci.invoice_number IS NULL
""")


In [21]:
only_in_gl.shape

(3900, 21)

In [22]:
only_in_pr.shape

(4322, 21)

In [23]:

#Combine the two views into one DataFrame
only_gl_or_pr_entry = duckdb.sql("""
    SELECT *
    FROM only_in_gl
    UNION ALL
    SELECT *
    FROM only_in_pr
""")

In [24]:
gl_common = duckdb.sql(""" 
SELECT gl.*
FROM gl as gl where gl."Invoice No" In (select * from common_invoices)""")

In [25]:
gl_common.shape

(84193, 10)

In [26]:
pr_common = duckdb.sql("""
SELECT pr.*
FROM pr_filtered pr

where pr."Document Number (PR)" In (select * from common_invoices) """)

In [27]:
pr_common.shape

(83904, 8)

In [28]:
pr_duplicates = duckdb.sql("""
    
    SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
            coalesce(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "IGST (PR)",
             coalesce(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)),0) as  "CGST (PR)",
             coalesce(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "SGST (PR)",
         CAST(REPLACE("Total Tax (PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
       FROM pr_ranked
    WHERE rn >= 2
""")


# Add duplicates with appropriate remarks
duplicates_df = duckdb.sql("""
    SELECT
       NULL AS "Supplier GSTIN/UIN",
       NULL AS "MY GSTIN",
       NULL AS "Invoice No", 
       NULL AS "Invoice Date", 
       NULL as "IGST (GL)",
       Null as "CGST (GL)",
       Null as "SGST (GL)",
       NULL AS "total Tax(GL)",
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)", 
        "Document Number (PR)",
        "Document Date (PR)", 
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        "Total Tax(PR)", 
        null as "Remark",
        null as "HL Remark",
        Null as "Matching keys",
       'Duplicate Entry In PR' AS "Remarks",
       Null as "Remarks_1"

    FROM pr_duplicates
""")


# Perform the full outer join and add remarks
result_1 = duckdb.sql("""
    SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark",
        gl_file."HL Remark",
        'SUPPLIER GSTIN-MY GSTIN-INVOICE NO-INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'All Match' as 'Remarks-1'
    FROM gl_common as gl_file
    FULL OUTER JOIN pr_common as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)"
""")

In [29]:
result_1.shape

(97021, 21)

In [30]:
result_1.df().to_csv('result_1.csv', index = False)

In [31]:
matched_1 = duckdb.sql( """select * from result_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [32]:
# matched_1.shape

In [33]:
matched_1.df().to_csv('matched_1.csv', index= False)

In [34]:
only_gl_or_pr_entry.df().to_csv('only_in_pr_gl.csv', index = False)

In [35]:
duplicates_df.df().to_csv('Duplicates.csv', index = False)

In [36]:
result_1 = pd.read_csv('result_1.csv')

In [37]:
pr_2 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [38]:
pr_2.shape

(12821, 8)

In [39]:
gl_2 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark"
 from result_1
 where  "Supplier GSTIN (PR)" is null """)

In [40]:
gl_2.shape

(13117, 10)

In [41]:
result_2 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)", 
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)", 
        pr_filtered."SGST (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."Total Tax(PR)",
        gl_file."Remark" as "Remark" ,
        gl_file."HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not match' as "Remarks_1"
    FROM gl_2 as gl_file
    FULL OUTER JOIN pr_2 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
""")


In [42]:
result_2.df().to_csv('result_2.csv', index = False)

In [43]:
matched_2 = duckdb.sql( """select * from result_2
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)


In [44]:
matched_2.df().to_csv('matched_2.csv', index= False)

In [45]:
result_2 = pd.read_csv('result_2.csv')

In [46]:
pr_2_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
         "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2
    WHERE "Supplier GSTIN/UIN" is null""")

In [47]:
gl_2_1 = duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_2
 where  "Supplier GSTIN (PR)" is null """)

In [48]:
result_2_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark",
        gl_file."HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO(Fuzzy)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not Match' as "Remarks_1"
    FROM gl_2_1 as gl_file
    FULL OUTER JOIN pr_2_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
    REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
     """)

In [49]:
result_2_1.df().to_csv('result_2_1.csv', index = False)

In [50]:
matched_2_1 = duckdb.sql( """select * from result_2_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )



In [51]:
matched_2_1.df().to_csv('matched_2_1.csv', index= False)

In [52]:
result_2_1 = pd.read_csv('result_2_1.csv')

In [53]:
pr_3 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2_1
    WHERE "Supplier GSTIN/UIN" is null  """)  

In [54]:
gl_3= duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" 
 from result_2_1
 where  "Supplier GSTIN (PR)" is null """)


In [55]:
result_3 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark" ,
        gl_file."HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN Not Match' as "Remarks_1"
    FROM gl_3 as gl_file
    FULL OUTER JOIN pr_3 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")


In [56]:
result_3.df().to_csv('result_3.csv', index = False)

In [57]:
matched_3 = duckdb.sql( """select * from result_3
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [58]:
# matched_3.shape

In [59]:
matched_3.df().to_csv('matched_3.csv', index= False)

In [60]:
result_3= pd.read_csv('result_3.csv')

In [61]:
pr_4= duckdb.sql(""" 
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)", 
        "CGST (PR)",  
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_3
    WHERE "Supplier GSTIN/UIN" is null  """)

In [62]:
 gl_4 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" ,
 from result_3
 where  "Supplier GSTIN (PR)" is null """)

In [63]:
result_4 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date Not match' as "Remarks_1"
    FROM gl_4 as gl_file
    FULL OUTER JOIN pr_4 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    
""")


In [64]:
result_4.df().to_csv('result_4.csv', index = False)

In [65]:
matched_4 = duckdb.sql( """select * from result_4
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [66]:
matched_4.df().to_csv('matched_4.csv', index= False)

In [67]:
result_4 = pd.read_csv('result_4.csv')

In [68]:
pr_4_1  =duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4
    WHERE "Supplier GSTIN/UIN" is null  """)

In [69]:
 gl_4_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark" ,
    "HL Remark",
 from result_4
 where  "Supplier GSTIN (PR)" is null """)

In [70]:
result_4_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_4_1 as gl_file
    FULL OUTER JOIN pr_4_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
      
""")


In [71]:
result_4_1.df().to_csv('result_4_1.csv', index= False)

In [72]:
matched_4_1 = duckdb.sql( """select * from result_4_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [73]:
matched_4_1.df().to_csv('matched_4_1.csv', index= False)

In [74]:
result_4_1 = pd.read_csv('result_4_1.csv')

In [75]:
pr_5 = duckdb.sql("""
         SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [76]:
 gl_5 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark",
 from result_4_1
 where  "Supplier GSTIN (PR)" is null """)

In [77]:
result_5 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."Total Tax(GL)",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN Not Match' as "Remarks_1"
    FROM gl_5 as gl_file
    FULL OUTER JOIN pr_5 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")

In [78]:
result_5.df().to_csv('result_5.csv', index= False)

In [79]:
matched_5 = duckdb.sql( """select * from result_5
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [80]:
matched_5.df().to_csv('matched_5.csv', index= False)

In [81]:
result_5= pd.read_csv('result_5.csv')

In [82]:
 pr_6 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_5
    WHERE "Supplier GSTIN/UIN" is null """)

In [83]:
gl_6 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
      "IGST (GL)",
      "CGST (GL)",
      "SGST (GL)",
      "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_5
 where  "Supplier GSTIN (PR)" is null """)

In [84]:
result_6 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and Date not match' as "Remarks_1"
    FROM gl_6 as gl_file
    FULL OUTER JOIN pr_6 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    
""")


In [85]:
result_6.df().to_csv('result_6.csv', index= False)

In [86]:
matched_6 = duckdb.sql( """select * from result_6
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [87]:
matched_6.df().to_csv('matched_6.csv', index= False)

In [88]:
result_6 = pd.read_csv('result_6.csv')

In [89]:
 pr_6_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6
    WHERE "Supplier GSTIN/UIN" is null """)

In [90]:
 gl_6_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6
 where  "Supplier GSTIN (PR)" is null """)

In [91]:
result_6_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
         pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Suppliers GSTIN and Date not match' as "Remarks_1"
    FROM gl_6_1 as gl_file
    FULL OUTER JOIN pr_6_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [92]:
result_6_1.df().to_csv('result_6_1.csv',index= False)

In [93]:
matched_6_1 = duckdb.sql( """select * from result_6_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [94]:
matched_6_1.df().to_csv('matched_6_1.csv', index= False)

In [95]:
result_6_1 = pd.read_csv('result_6_1.csv')

In [96]:
 pr_7= duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6_1
    WHERE "Supplier GSTIN/UIN" is null  """)

In [97]:
# pr_7.shape

In [98]:
 gl_7 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6_1
 where  "Supplier GSTIN (PR)" is null """)

In [99]:
result_7 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and MY GSTIN not match' as "Remarks_1"
    FROM gl_7 as gl_file
    FULL OUTER JOIN pr_7 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")

In [100]:
result_7.df().to_csv('result_7.csv',index= False)

In [101]:
matched_7 = duckdb.sql( """select * from result_7
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [102]:
matched_7.df().to_csv('matched_7.csv', index= False)

In [103]:
result_7= pd.read_csv('result_7.csv')

In [104]:
 pr_8 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_7
    WHERE "Supplier GSTIN/UIN" is null  """)

In [105]:
gl_8 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
   "HL Remark" 
 from result_7
 where  "Supplier GSTIN (PR)" is null """)

In [106]:
result_8 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8 as gl_file
    FULL OUTER JOIN pr_8 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
""")


In [107]:
result_8.df().to_csv('result_8.csv',index= False)

In [108]:
matched_8 = duckdb.sql( """select * from result_8
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [109]:
matched_8.df().to_csv('matched_8.csv', index= False)

In [110]:
result_8 = pd.read_csv('result_8.csv')

In [111]:
 pr_8_1  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8
    WHERE "Supplier GSTIN/UIN" is null """)

In [112]:
 gl_8_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_8
 where  "Supplier GSTIN (PR)" is null """)

In [113]:
result_8_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8_1 as gl_file
    FULL OUTER JOIN pr_8_1 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [114]:
result_8_1.df().to_csv('result_8_1.csv',index= False)

In [115]:
matched_8_1 = duckdb.sql( """select * from result_8_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [116]:
result_8_1 = pd.read_csv('result_8_1.csv')

In [117]:
only_gl_or_pr_entry = pd.read_csv('only_in_pr_gl.csv')

In [118]:
 pr_9  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8_1
    WHERE "Supplier GSTIN/UIN" is null 
    union all 
    select
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    from  only_gl_or_pr_entry
    WHERE "Supplier GSTIN/UIN" is null 
    
    """)

In [119]:
gl_9 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
  from result_8_1
  where  "Supplier GSTIN (PR)" is null
  union all
  select
  "Supplier GSTIN/UIN",
    "MY GSTIN",
    "Invoice No", 
    "Invoice Date",
    "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from only_gl_or_pr_entry
 where  "Supplier GSTIN (PR)" is null""" )

In [120]:
result_9 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER GSTIN ,MY GSTIN and Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Invoice Number and Date Not Match' as "Remarks_1"
    FROM gl_9 as gl_file
    FULL OUTER JOIN pr_9 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
""")


In [121]:
result_9.df().to_csv('result_9.csv',index= False)

In [122]:
matched_9 = duckdb.sql( """select * from result_9
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [123]:
result_9= pd.read_csv('result_9.csv')

In [124]:
 pr_10  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_9
    WHERE "Supplier GSTIN/UIN" is null """)

In [125]:
 gl_10 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_9
 where  "Supplier GSTIN (PR)" is null """)

In [126]:
result_10 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'MY GSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_10 as gl_file
    FULL OUTER JOIN pr_10 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        """)


In [127]:
result_10.df().to_csv('result_10.csv',index= False)

In [128]:
matched_10 = duckdb.sql( """select * from result_10
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [129]:
result_10= pd.read_csv('result_10.csv')

In [130]:
pr_11  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_10
    WHERE "Supplier GSTIN/UIN" is null """)

In [131]:
gl_11 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_10
 where  "Supplier GSTIN (PR)" is null """)

In [132]:
result_11 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'SupplierGSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_11 as gl_file
    FULL OUTER JOIN pr_11 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN"  = pr_filtered."My GSTIN (PR)"
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [133]:
result_11.df().to_csv('result_11.csv',index= False)

In [134]:
matched_11 = duckdb.sql( """select * from result_11
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [135]:
result_11= pd.read_csv('result_11.csv')

In [136]:
pr_12  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_11
    WHERE "Supplier GSTIN/UIN" is null """)

In [137]:
gl_12 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_11
 where  "Supplier GSTIN (PR)" is null """)

In [138]:
result_12 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_12 as gl_file
    FULL OUTER JOIN pr_12 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [139]:
result_12.df().to_csv('result_12.csv',index= False)

In [140]:
matched_12 = duckdb.sql( """select * from result_12
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [141]:
result_12 = pd.read_csv('result_12.csv')

In [142]:
pr_13  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_12
    WHERE "Supplier GSTIN/UIN" is null """)

In [143]:
gl_13 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_11
 where  "Supplier GSTIN (PR)" is null """)

In [144]:
result_13 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
         'BUYER GSTIN/PAN, INOVICE NO, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN(PAN),MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_13 as gl_file
    FULL OUTER JOIN pr_13 as pr_filtered
    ON 
     (
            gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)" 
            OR SUBSTRING(gl_file."MY GSTIN", 3, 10) = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10)
        )
AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [145]:
result_13.df().to_csv('result_13.csv',index= False)

In [146]:
matched_13 = duckdb.sql( """select * from result_13
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [147]:
matched_13.shape

(9474, 21)

In [148]:
combined_sheet = duckdb.sql("""
                    select   * from matched_1
                    union all
                    select   * from matched_2
                    union all 
                    select   * from matched_2_1
                    union all
                    select   * from matched_3 
                    union all 
                    select   * from matched_4
                    union all 
                    select   * from matched_4_1
                    union all
                    select * from matched_5
                    union all
                    select * from matched_6
                    union all 
                    select * from matched_6_1
                    union all 
                    select * from matched_7
                    union all
                    select * from matched_8
                    union all 
                    select * from matched_8_1
                    union all
                    select * from matched_9
                    union all
                    select * from matched_10
                    union all
                    select * from matched_11
                    union all
                    select * from matched_12
                    union all
                    select * from result_13
                    
                     """)


In [149]:
combined_sheet.df().to_csv('combined_sheet.csv', index = False)

In [150]:
duplicates_df = pd.read_csv("Duplicates.csv")

In [151]:
final_sheet = duckdb.sql("""
    select * from combined_sheet
    union all
    select * from duplicates_df
""")

In [152]:
final_sheet.df().to_csv('final_sheet.csv', index= False)